In [ ]:
import itertools

def at_least_sat(n, variables, condition):
    return '|'.join(['&'.join([condition.format(v=variable) for variable in and_variables]) for and_variables in itertools.combinations(variables, n)])
    

buf_size = 2
num_nodes = 2
quorum_size = num_nodes//2+1
log_size = 5
queue_size = 2
max_round = 3

prologue_code = f"""// Rabia Algorithm


pta

const NUM_NODES = {num_nodes};
const QUORUM_SIZE = {quorum_size};
const BUF_SIZE = {buf_size};
const MAX_TS = 2;
const QUEUE_SIZE = {queue_size};
const LOG_SIZE = {log_size};
const MAX_ROUND = {max_round};

const p1_stage = 0;
const p2s_stage = 1;
const p2v_stage = 2;
const decided = 3;

const cmd_request = 0;
const cmd_proposal = 1;
const cmd_state = 2;
const cmd_vote = 3;
const NUM_CMD_TYPES = 4;

const INVALID=-2;
const BOT = -1;

"""

# Packet Types:
# - <Request, cmd: Command>
# - <Proposal, q: Command>
# - <State, round, state: Command|bot>
# - <Vote, round, vote: Command|bot>
wire_code = f"""
module wire01""" + ''.join([f"""
    // pkt slot {i}
    w01_pkt_valid_{i} : bool init false;
    w01_pkt_type_{i} : [0..NUM_CMD_TYPES] init 0;
    w01_pkt_seq_{i} : [0..LOG_SIZE] init 0;
    w01_pkt_round_{i} : [0..MAX_ROUND] init 0;
    w01_pkt_cmd_ts_{i} : [-2..MAX_TS] init 0;""" for i in range(buf_size)]) + ''.join([f"""
    [n0_send_boradcast] n0_send_ready=true & n0_send_is_broadcast=true & w01_pkt_valid_{i}=false & n0_send_ready=true -> \
        (w01_pkt_valid_{i}'=true) & (w01_pkt_type_{i}'=n0_send_type) & (w01_pkt_seq_{i}'=n0_send_seq) & (w01_pkt_round_{i}'=n0_send_round) & (w01_pkt_cmd_ts_{i}'=n0_send_cmd_ts);
    [send_01_{i}] n0_send_ready=true & n0_send_is_broadcast=false & w01_pkt_valid_{i} = false & n0_send_ready=true -> \
        (w01_pkt_valid_{i}'=true) & (w01_pkt_type_{i}'=n0_send_type) & (w01_pkt_seq_{i}'=n0_send_seq) & (w01_pkt_round_{i}'=n0_send_round) & (w01_pkt_cmd_ts_{i}'=n0_send_cmd_ts);
    [recv_01_{i}] n0_recv_ready=false & w01_pkt_valid_{i} = true -> (w01_pkt_valid_{i}'=false);""" for i in range(buf_size)]) + """

endmodule
"""

node_code = f"""
module node0
    // states
	n0_stage : [0..4] init p1_stage;
    /// p1 stage
    n0_seq : [0..LOG_SIZE] init 0;""" + ''.join([f"""
    n0_pq_valid_{i} : bool init false;
    n0_pq_ts_{i} : [0..MAX_TS] init 0;""" for i in range(queue_size)]) + ''.join([f"""
    n0_log_ts_{i} : [-2..LOG_SIZE] init INVALID; """ for i in range(log_size)]) + """
    /// p2 stage""" + ''.join([f"""
    n0_round_{i} : [0..MAX_ROUND] init 0; 
    n0_my_proposal_{i} : [0..MAX_TS] init 0;
    n0_state_ts_{i} : [-2..MAX_TS] init INVALID;
    n0_vote_ts_{i} : [-2..MAX_TS] init INVALID;""" for i in range(log_size)]) + ''.join([f"""
    n0_proposal_{i}_n{nid} : [-2..MAX_TS] init INVALID;
    n0_state_ts_{i}_n{nid} : [-2..MAX_TS] init INVALID;
    n0_vote_ts_{i}_n{nid} : [-2..MAX_TS] init INVALID;""" for i, nid in itertools.product(range(log_size), range(num_nodes))]) + """

    // pkt to send
    n0_send_ready : bool init false; // ready means not yet sent
    n0_send_is_broadcast : bool init false;
    n0_send_type : [0..NUM_CMD_TYPES] init 0;
    n0_send_seq : [0..LOG_SIZE] init 0;
    n0_send_round : [0..MAX_ROUND] init 0;
    n0_send_cmd_ts : [-2..MAX_TS] init 0;

    // pkt received
    n0_recv_ready : bool init false; // ready means received
    n0_recv_from : [0..NUM_NODES] init 0;
    n0_recv_type : [0..NUM_CMD_TYPES] init 0;
    n0_recv_seq : [0..LOG_SIZE] init 0;
    n0_recv_round : [0..MAX_ROUND] init 0;
    n0_recv_cmd_ts : [-2..MAX_TS] init 0;""" + """
    
    // Try on each queue slot and execute if the slot has min ts""" + ''.join([f"""
    [n0_propose_next_command_for_{seq}_{i}] n0_stage=p1_stage & n0_send_ready=false & n0_seq={seq} & n0_pq_valid_{i}=true & {'&'.join([f"(!n0_pq_valid_{j}|n0_pq_ts_{j}>=n0_pq_ts_{i})" for j in range(queue_size)])} -> \
        (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_proposal) & (n0_send_cmd_ts'=n0_pq_ts_{i}) & (n0_proposal_{seq}_n0'=n0_pq_ts_{i});""" for i, seq in itertools.product(range(queue_size), range(log_size))]) + """
    
    // Process when a Proposal pkt is ready.
    """ + ''.join([f"""
    [n0_process_proposal_from_n{nid}_{i}] n0_stage=p1_stage & n0_recv_ready=true & n0_recv_from={nid} & n0_recv_type=cmd_proposal -> \
        (n0_proposal_{i}_n{nid}'=n0_recv_cmd_ts);""" for i, nid in itertools.product(range(log_size), range(num_nodes))]) + """
    // TODO: here is a hack using n0_proposal_i_nnid=n0_proposal_i_nnid for two nodes scenario.""" + ''.join([f"""
    [n0_process_proposal_assign_state_{i}_from_{nid}] n0_stage=p1_stage & n0_send_ready=false & ({at_least_sat(quorum_size, [f"n0_proposal_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")}) & n0_proposal_{i}_n0=n0_proposal_{i}_n1 -> \
        (n0_state_ts_{i}'=n0_proposal_{i}_n{nid}) & (n0_stage'=p2s_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_state) & (n0_send_cmd_ts'=n0_proposal_{i}_n{nid});
    [n0_process_proposal_assign_state_{i}_from_{nid}] n0_stage=p1_stage & ({at_least_sat(quorum_size, [f"n0_proposal_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")}) & n0_proposal_{i}_n0!=n0_proposal_{i}_n1 -> \
        (n0_state_ts_{i}'=BOT) & (n0_stage'=p2s_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_state) & (n0_send_cmd_ts'=BOT);""" for i, nid in itertools.product(range(log_size), range(num_nodes))]) + """

    // Process when a State pkt is ready.
    """ + ''.join([f"""
    [n0_process_state_from_n{nid}_{i}] n0_stage=p2s_stage & n0_recv_ready=true & n0_recv_from={nid} & n0_recv_type=cmd_state -> \
        (n0_state_ts_{i}_n{nid}'=n0_recv_cmd_ts);""" for i, nid in itertools.product(range(log_size), range(num_nodes))]) + """
    // TODO: here is a hack using n0_state_ts_i_0=n0_state_ts_i_1 for two nodes scenario.""" + ''.join([f"""
    [n0_process_state_assign_state_{i}_from_{nid}] n0_stage=p2s_stage & n0_send_ready=false & ({at_least_sat(quorum_size, [f"n0_state_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")}) & n0_state_ts_{i}_n0=n0_state_ts_{i}_n1 -> \
        (n0_vote_ts_{i}'=n0_state_ts_{i}_n{nid}) & (n0_stage'=p2v_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_vote) & (n0_send_cmd_ts'=n0_state_ts_{i}_n{nid});
    [n0_process_state_assign_state_{i}_from_{nid}] n0_stage=p2s_stage & ({at_least_sat(quorum_size, [f"n0_state_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")}) & (n0_state_ts_{i}_n0!=n0_state_ts_{i}_n1) -> \
        (n0_vote_ts_{i}'=BOT) & (n0_stage'=p2v_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_vote) & (n0_send_cmd_ts'=BOT);""" for i, nid in itertools.product(range(log_size), range(num_nodes))]) + """

    // Process when a Vote pkt is ready.
    """ + ''.join([f"""
    [n0_process_vote_from_n{nid}_{i}] n0_stage=p2v_stage & n0_recv_ready=true & n0_recv_from={nid} & n0_recv_type=cmd_vote -> \
        (n0_vote_ts_{i}_n{nid}'=n0_recv_cmd_ts);""" for i, nid in itertools.product(range(log_size), range(num_nodes))]) + """
    // TODO: here is a hack using n0_vote_ts_i_n0=n0_vote_ts_i_n1 for two nodes scenario.""" + ''.join([f"""
    [n0_process_vote_from_n{i}_from_{nid}] n0_stage=p2v_stage & n0_send_ready=false & {at_least_sat(quorum_size, [f"n0_vote_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")} & (n0_vote_ts_{i}_n0=n0_vote_ts_{i}_n1 & n0_vote_ts_{i}_n0!=BOT) -> \
        (n0_log_ts_{i}'=n0_state_ts_{i}_n{nid}) & (n0_stage'=p1_stage);
    [n0_process_vote_from_n{i}_from_{nid}] n0_stage=p2v_stage & {at_least_sat(quorum_size, [f"n0_vote_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")} & (n0_vote_ts_{i}_n0!=n0_vote_ts_{i}_n1 & n0_vote_ts_{i}_n0!=BOT) -> \
        (n0_vote_ts_{i}'=n0_state_ts_{i}_n0) & (n0_stage'=p2s_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_state) & (n0_send_cmd_ts'=n0_state_ts_{i}_n0) & {' & '.join([f"(n0_state_ts_{i}_n{j}'=INVALID)&(n0_vote_ts_{i}_n{j}'=INVALID)" for j in range(num_nodes)])};
    [n0_process_vote_from_n{i}_from_{nid}] n0_stage=p2v_stage & {at_least_sat(quorum_size, [f"n0_vote_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")} & (n0_vote_ts_{i}_n0!=n0_vote_ts_{i}_n1 & n0_vote_ts_{i}_n1!=BOT) -> \
        (n0_vote_ts_{i}'=n0_state_ts_{i}_n1) & (n0_stage'=p2s_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_state) & (n0_send_cmd_ts'=n0_state_ts_{i}_n1) & {' & '.join([f"(n0_state_ts_{i}_n{j}'=INVALID)&(n0_vote_ts_{i}_n{j}'=INVALID)" for j in range(num_nodes)])};
    [n0_process_vote_from_n{i}_from_{nid}] n0_stage=p2v_stage & {at_least_sat(quorum_size, [f"n0_vote_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")} & (n0_vote_ts_{i}_n0=n0_vote_ts_{i}_n1 & n0_vote_ts_{i}_n1=BOT & n0_state_ts_{i}_n0!=INVALID & n0_state_ts_{i}_n0!=BOT) -> \
        (n0_vote_ts_{i}'=BOT) & (n0_stage'=p2s_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_state) & (n0_send_cmd_ts'=BOT) & {' & '.join([f"(n0_state_ts_{i}_n{j}'=INVALID)&(n0_vote_ts_{i}_n{j}'=INVALID)" for j in range(num_nodes)])};
    [n0_process_vote_from_n{i}_from_{nid}] n0_stage=p2v_stage & {at_least_sat(quorum_size, [f"n0_vote_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")} & (n0_vote_ts_{i}_n0=n0_vote_ts_{i}_n1 & n0_vote_ts_{i}_n1=BOT & n0_state_ts_{i}_n0!=INVALID & n0_state_ts_{i}_n0!=BOT) -> \
        (n0_vote_ts_{i}'=n0_state_ts_{i}_n0) & (n0_stage'=p2s_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_state) & (n0_send_cmd_ts'=n0_state_ts_{i}_n0) & {' & '.join([f"(n0_state_ts_{i}_n{j}'=INVALID)&(n0_vote_ts_{i}_n{j}'=INVALID)" for j in range(num_nodes)])};
    [n0_process_vote_from_n{i}_from_{nid}] n0_stage=p2v_stage & {at_least_sat(quorum_size, [f"n0_vote_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")} & (n0_vote_ts_{i}_n0=n0_vote_ts_{i}_n1 & n0_vote_ts_{i}_n1=BOT & n0_state_ts_{i}_n1!=INVALID & n0_state_ts_{i}_n1!=BOT) -> \
        (n0_vote_ts_{i}'=BOT) & (n0_stage'=p2s_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_state) & (n0_send_cmd_ts'=BOT) & {' & '.join([f"(n0_state_ts_{i}_n{j}'=INVALID)&(n0_vote_ts_{i}_n{j}'=INVALID)" for j in range(num_nodes)])};
    [n0_process_vote_from_n{i}_from_{nid}] n0_stage=p2v_stage & {at_least_sat(quorum_size, [f"n0_vote_ts_{i}_n{j}" for j in range(num_nodes)], "{v}!=INVALID")} & (n0_vote_ts_{i}_n0=n0_vote_ts_{i}_n1 & n0_vote_ts_{i}_n1=BOT & n0_state_ts_{i}_n1!=INVALID & n0_state_ts_{i}_n1!=BOT) -> \
        (n0_vote_ts_{i}'=n0_state_ts_{i}_n1) & (n0_stage'=p2s_stage) & (n0_send_ready'=true) & (n0_send_is_broadcast'=true) & (n0_send_type'=cmd_state) & (n0_send_cmd_ts'=n0_state_ts_{i}_n1) & {' & '.join([f"(n0_state_ts_{i}_n{j}'=INVALID)&(n0_vote_ts_{i}_n{j}'=INVALID)" for j in range(num_nodes)])};""" for i, nid in itertools.product(range(log_size), range(num_nodes))]) + """

    // Send when a pkt is ready; Recv when a pkt is ready.""" + ''.join([f"""
    [recv_01_{i}] n0_recv_ready=false & w01_pkt_valid_{i} = true -> \
        (n0_recv_ready'=true) & (n0_recv_from'={i}) & (n0_recv_type'=w01_pkt_type_{i}) & (n0_recv_round'=w01_pkt_round_{i}) & (n0_recv_cmd_ts'=w01_pkt_cmd_ts_{i});
    [n0_send_boradcast] n0_send_ready=true & n0_send_is_broadcast=true & w01_pkt_valid_{i} = false -> (n0_send_ready'=false);    
    [send_01_{i}] n0_send_ready=true & n0_send_is_broadcast=false & w01_pkt_valid_{i} = false -> (n0_send_ready'=false);""" for i in range(buf_size)]) + """

endmodule
"""

node_variables_list = [
    [
        f"n{nid}_stage", f"n{nid}_seq", 
        *[f"n{nid}_pq_valid_{i}" for i in range(queue_size)], 
        *[f"n{nid}_pq_ts_{i}" for i in range(queue_size)], 
        *[f"n{nid}_log_ts_{i}" for i in range(log_size)], 
        *[f"n{nid}_round_{i}" for i in range(log_size)],
        *[f"n{nid}_my_proposal_{i}" for i in range(log_size)],
        *[f"n{nid}_state_ts_{i}" for i in range(log_size)],
        *[f"n{nid}_vote_ts_{i}" for i in range(log_size)],
        *[f"n{nid}_proposal_{i}_n{j}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"n{nid}_state_ts_{i}_n{j}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"n{nid}_vote_ts_{i}_n{j}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"n{nid}_send_ready", f"n{nid}_send_is_broadcast", f"n{nid}_send_type", f"n{nid}_send_seq", f"n{nid}_send_round", f"n{nid}_send_cmd_ts"],
        *[f"n{nid}_recv_ready", f"n{nid}_recv_from", f"n{nid}_recv_type", f"n{nid}_recv_seq", f"n{nid}_recv_round", f"n{nid}_recv_cmd_ts"],
        *[f"n{nid}_propose_next_command_for_{seq}_{i}" for i, seq in itertools.product(range(queue_size), range(log_size))],
        *[f"n{nid}_process_proposal_from_n{j}_{i}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"n{nid}_process_proposal_assign_state_{i}_from_{j}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"n{nid}_process_state_from_n{j}_{i}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"n{nid}_process_state_assign_state_{i}_from_{j}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"n{nid}_process_vote_from_n{j}_{i}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"n{nid}_process_vote_from_n{i}_from_{j}" for i, j in itertools.product(range(log_size), range(num_nodes))],
        *[f"recv_{nid}{j}_{i}" for i, j in itertools.product(range(buf_size), range(num_nodes))],
        f"n{nid}_send_boradcast", 
        *[f"send_{nid}{j}_{i}" for i, j in itertools.product(range(buf_size), range(num_nodes))],
    ]
    for nid in range(num_nodes)
]

wire_variables_list = {
    (i, j): [
        f"n{i}_send_broadcast", 
    ] + list(itertools.chain(*[[
        f"w{i}{j}_pkt_valid_{bid}", f"w{i}{j}_pkt_type_{bid}", f"w{i}{j}_pkt_seq_{bid}", f"w{i}{j}_pkt_round_{bid}", f"w{i}{j}_pkt_cmd_ts_{bid}", 
        f"send_{i}{j}_{bid}", f"recv_{i}{j}_{bid}"] for bid in range(buf_size)]))
    for i, j in itertools.product(range(num_nodes), range(num_nodes))
}

module_rename_codes = [
    f"module node1 = node0[" + ','.join([f'{name0}={name1}' for name0, name1 in zip(node_variables_list[0], node_variables_list[1])]) + f"] endmodule",
    f"module wire10 = wire01[" + ','.join([f'{name0}={name1}' for name0, name1 in zip(wire_variables_list[(0,1)], wire_variables_list[(1,0)])]) + f"] endmodule",
]

epilogue_code = """
// rewards (to calculate expected number of steps)
rewards "steps"
    true : 1;
endrewards

// label "elected_1" = role_1=LEADER;
// label "elected_2" = role_2=LEADER;
"""

code = ''.join([prologue_code, wire_code, node_code, '\n'.join(module_rename_codes), epilogue_code])
# print(code)
open('rabia.nm', 'w').write(code)